In [5]:
import tensorflow as tf
x = tf.constant ([[3,5,7],
                 [4,6,8]])
y = tf.reshape(x,[3,2])
with tf.Session() as sess:
    print (y.eval())

[[3 5]
 [7 4]
 [6 8]]


In [6]:
y = tf.reshape(x,[3,2])[1,:]
with tf.Session() as sess:
    print (y.eval())

[7 4]


### Variables

In [15]:
#Build Computational Graph
def forward_pass(w,x):
    return tf.matmul(w,x)

def train_loop(x, niter=5):
    with tf.variable_scope("model",reuse = tf.AUTO_REUSE):
        w = tf.get_variable("weights",
                           shape = (1,2), #1 x 2 matrix
                           initializer = tf.truncated_normal_initializer(), #initializes to random numbers with gaussian normal distribution with 0 mean. Truncates the distribution to avoid extreme outliers
                           trainable = True) #trainable => you can train this during evaluation
    preds = []
    for k in range (niter):
        preds.append(forward_pass(w,x))
        w = w+0.1 #"gradient update"
    return preds

#Run Session
with tf.Session() as sess:
    preds = train_loop(tf.con stant([[3.2,5.1,7.2],[4.3,6.2,8.3]])) #2 x 3 matrix
    tf.global_variables_initializer().run() #initialize all variables
    for i in range(len(preds)):
        print("{}:{}".format(i,preds[i].eval()))
        

0:[[3.6920192 5.75547   8.036126 ]]
1:[[4.4420195 6.88547   9.586126 ]]
2:[[ 5.1920195  8.0154705 11.1361265]]
3:[[ 5.9420195  9.14547   12.686127 ]]
4:[[ 6.6920195 10.27547   14.236126 ]]


In [18]:
#Placeholder Examples: Placeholders allow you to feed in values, such as by reading from a text file
import tensorflow as tf

a = tf.placeholder("float", None)
b = a*4
print (a)
with tf.Session() as sess:
    print(sess.run(b, feed_dict={a:[1,2,3]}))

Tensor("Placeholder:0", dtype=float32)
[ 4.  8. 12.]


### Debugging Tensorflow Programs

In [1]:
# tf.Print() allows printing of values
# you can change the log level using tf.logging.set_verbosity(tf.logging.INFO). This specific level will show many more details than typical log
import sys
sys.modules[__name__].__dict__.clear()
import tensorflow as tf

tf.reset_default_graph()

def some_method (a,b):
    b = tf.cast(b,tf.float32)
    s = (a/b) #oops, NaN
    print_ab = tf.Print(s,[a,b])
    return tf.sqrt(tf.matmul(s,tf.transpose(s)))

with tf.Session() as sess:
    fake_a = tf.constant([[5.,3.,7.1],[2.3,4.1,4.8]])
    fake_b = tf.constant([[2,0,5],[2,8,7]])
    print (sess.run(some_method(fake_a,fake_b)))

/Users/johnleonard/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[[      inf       inf]
 [      inf 1.4336528]]


In [ ]:
## you can also debug using command line "python <file> --debug" to get an interactive debugger

### Estimator API

Estimators wrap up a large amount of boilerplate code on top of the model itself
Estimators allow quick modeling, chechpointing, out-of-memory dataset, train/eval/monitoring, hyper-parameter tuning on ML-egning, production
base clasE: "tf.estimator.model", where model = 
    <ol> LinearRegressor, DNNRegressor, DNNLinearCombinedRegressor,LinearClassifier,DNNLinearCombinedClassifier, etc.

In [23]:
#Example: Predict price of property
import tensorflow as tf

# Now we write an input fxn
def train_input_fn():
    features = {"sq_footage": [1000, 2000, 3000, 1000, 2000, 3000],
                "type" : ["house","house","house","apt","apt","apt"]}
    labels = [500, 1000, 1500, 700, 1300, 1900]
    return features, labels

featcols = [
    tf.feature_column.numeric_column("sq_footage"),
    tf.feature_column.categorical_column_with_vocabulary_list("type", ["house","apt"]) #This performs one-hot encoding assigning 0 or 1 to the classes
]

model = tf.estimator.LinearRegressor(featcols,"outdir")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'outdir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10b79d860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [24]:
#train the model
model.train(train_input_fn, steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into outdir/model.ckpt.
INFO:tensorflow:loss = 9290000.0, step = 1
INFO:tensorflow:Saving checkpoints for 100 into outdir/model.ckpt.
INFO:tensorflow:Loss for final step: 144267.48.


KeyError: '_oh'

In [25]:
#predict
def predict_input_fn():
    features = {"sq_footage":[1500, 1800],
               "type": ["house","apt"]}
    return features

predictions = model.predict(predict_input_fn)
predictions

KeyError: '_oh'

### Train on Large Datasets with Datasets API
allows you to load data as a bach